In [ ]:
!pip install setfit datasets

In [ ]:
import pandas as pd
from datasets import Dataset
from setfit import SetFitModel, SetFitTrainer

In [ ]:
trn = "C:/Users/satra/Downloads/jigsaw-agile-community-rules/train.csv"
tst = "C:/Users/satra/Downloads/jigsaw-agile-community-rules/test.csv"
df_trn = pd.read_csv(trn)
# df_trn = df_trn.sample(frac=.01, random_state=42).reset_index(drop=True)
df_tst = pd.read_csv(tst)

trn_rows = []
for idx, row in df_trn.iterrows():
  for col in ['positive_example_1', 'positive_example_2']:
    trn_rows.append({'text': 'Subrreddit: '+ row['subreddit'] + ' [SEP] ' + 'Rule: ' + row['rule'] + ' [SEP] ' + 'Comment: ' + row[col], 'label': 1})
  for col in ['negative_example_1', 'negative_example_2']:
    trn_rows.append({'text': 'Subrreddit: '+ row['subreddit'] + ' [SEP] ' + 'Rule: ' + row['rule'] + ' [SEP] ' + 'Comment: ' + row[col], 'label': 0})

val_rows = []
for idx, row in df_trn.iterrows():
  val_rows.append({'text': 'Subrreddit: '+ row['subreddit'] + ' [SEP] ' + 'Rule: ' + row['rule'] + ' [SEP] ' + 'Comment: ' + row['body'], 'label': row['rule_violation']})

trn_df = pd.DataFrame(trn_rows)
trn_set = Dataset.from_pandas(trn_df)

val_df = pd.DataFrame(val_rows)
val_set = Dataset.from_pandas(val_df)

In [ ]:
modl = SetFitModel.from_pretrained('sentence-transformers/stsb-xlm-r-multilingual')
trnr = SetFitTrainer(model=modl,train_dataset=trn_set,eval_dataset=val_set,metric='accuracy',batch_size=8,num_epochs=1,num_iterations=8,learning_rate=1.e-5)

In [ ]:
sample_df = val_df.loc[val_df["label"] == 1, ['text']].sample(frac=.1, random_state=42)
sample_df.head(16)

In [ ]:
trnr.train()